In [1]:
import re
import time
import requests
from io import StringIO
from datetime import datetime, timedelta

from IPython.display import display
from boto.s3.key import Key
import pandas as pd
import boto3

/home/rmg/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
_metrics = "page_avg_scroll,page_avg_time,page_scroll_starts,page_total_time,page_uniques,page_views,page_views_loyal,page_views_quality"

def get_response(url, querystring, only_response=False):
    headers = {
        'cache-control': "no-cache"
    }
    response = requests.request(
        "GET", url, headers=headers, params=querystring
    )
    if only_response:
        return response
    
    return response.json() if response.ok else {'status': response.status_code}

def get_metrics(apikey, host, day):
    url = "http://api.chartbeat.com/query/v2/submit/page/"
    querystring = {
        "host": host,
        "apikey": apikey,
        "start": day,
        "end": day,
        "limit": 100000,
        "tz": "America/Lima",
        "dimensions": 'path',
        "metrics": _metrics,
        "referrer_type": 'social,search,direct,internal,links'
    }
    return get_response(url, querystring)

def get_status(query_id, apikey, host):
    url = "http://api.chartbeat.com/query/v2/status/"
    querystring = {
        "query_id": query_id,
        "apikey": apikey,
        "host": host
    } 
    return get_response(url, querystring)

def get_report(query_id, apikey, host):
    url = 'http://api.chartbeat.com/query/v2/fetch/'
    querystring = {
        "query_id": query_id,
        "apikey": apikey,
        "host": host
    }
    return get_response(url, querystring, only_response=True)

In [3]:
host = "gestion.pe"
apikey = "094d70f05bf0f49f7ba8126e7ee62c76"
# day = "2019-02-28"

In [4]:
def get_df(query_id, apikey, host, day):
    # Obtengo el response del reporte
    report = get_report(query_id, apikey, host)
    
    set_parameter_csv = {'sep': ',', 'low_memory': False}
    df = pd.read_csv(StringIO(report.text), **set_parameter_csv)
    
    cols = list(df.columns)
    if 'path' in cols:
        patron_articulo = re.compile(r'-\d{2,}$')
        df['is_article'] = df.path.apply(
            lambda _: 1 if patron_articulo.search(_) and len(_.split('/')[-1]) > 12 else 0
        )
        
        df = df[df['is_article'] == 1]
        df['article_id'] = df.path.apply(lambda _: _.split('-')[-1])
        del df['is_article']
        df['dia'] = day
        
    for col in _metrics.split(','):
        if col in cols:
            df[col].fillna(0, inplace=True)
            df[col] = df[col].astype('int')

    print(list(df.columns))
    # df = spark.createDataFrame(df)
    return df

In [5]:
start = datetime.strptime("2018-12-18", "%Y-%m-%d").date()
end = datetime.strptime("2019-01-18", "%Y-%m-%d").date()
set_parameter_csv = {
    'sep': ',',
    'low_memory': False
}

for dayy in (start + timedelta(days=_) for _ in range(0, (end - start).days + 1)):
    month = str(dayy.month).zfill(2)
    year = dayy.year
    dayy = str(dayy)
    query_response = get_metrics(apikey, host, dayy)
    query_id = query_response.get('query_id')
    
    try:
        time.sleep(3)
        df = get_df(query_id, apikey, host, dayy)
        print('CONNECCION RAPIDA')
    except Exception as e:
        print('/'*20, str(e))
        try:
            time.sleep(60)
            df = get_df(query_id, apikey, host, dayy)
            print("CONNECCION PROLONGADA")
        except Exception as e:
            print('='*20, str(e))       
    
    # df_.repartition(1).write.mode("overwrite").csv('s3n://charbeat.dev/elcomercio/raw/chartbeat_{0}.csv'.format(dayy), header=True)
    
    # SAVE IN LOCAL
    df.to_csv(dayy + '.csv', index=False)
    
    # UP S3 AWS
    bucket = 'charbeat-trafic'
    path_aws = 'gestion.pe/{}/{}/{}.csv'.format(year, month, dayy)
    csv_buffer = StringIO()
    
    df.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, path_aws).put(Body=csv_buffer.getvalue())
    print(dayy)
    
    #conn = boto3.s3.connect_to_region('us-east-1')  # or region of choice
    # ACCESO PUBLICO
    s3 = boto3.resource('s3')
    object_acl = s3.ObjectAcl(bucket, path_aws)
    response = object_acl.put(ACL='public-read')
    print('response: ',response)
    
    url_up = 'https://s3.amazonaws.com/{}/{}'.format(bucket, path_aws)
    df = pd.read_csv(url_up, **set_parameter_csv)
    display(df.head())
    display(df.isnull().sum())
    print("="*30)

['path', 'page_avg_scroll', 'page_avg_time', 'page_scroll_starts', 'page_total_time', 'page_uniques', 'page_views', 'page_views_loyal', 'page_views_quality', 'article_id', 'dia']
CONNECCION RAPIDA
2018-12-18
response:  {'ResponseMetadata': {'RequestId': '60EA6533398C157C', 'HostId': 'OJlZ4e1V54sY2A6deMrXNkKv0SaSKlXa2wNAyo9Iux9OBaf9Rw0neFks1jGYNXdxAtfueWBwZHs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'OJlZ4e1V54sY2A6deMrXNkKv0SaSKlXa2wNAyo9Iux9OBaf9Rw0neFks1jGYNXdxAtfueWBwZHs=', 'x-amz-request-id': '60EA6533398C157C', 'date': 'Fri, 08 Mar 2019 17:57:37 GMT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


,path,page_avg_scroll,page_avg_time,page_scroll_starts,page_total_time,page_uniques,page_views,page_views_loyal,page_views_quality,article_id,dia
0,archivo.gestion.pe/amp/panelg/8-inventos-tecno...,0,61,0,61,1,1,0,1,2205664,2018-12-18
1,archivo.gestion.pe/clinicas-lanzan-seguro-priv...,754,44,1,132,3,3,0,2,44,2018-12-18
2,archivo.gestion.pe/designan-feliciano-amado-co...,7,0,1,0,1,1,0,0,65,2018-12-18
3,archivo.gestion.pe/economia-peru-superara-chil...,1607,67,1,67,1,1,0,1,2050,2018-12-18
4,archivo.gestion.pe/economia/22-monedas-colecci...,1024,15,1,15,1,1,0,1,2147420,2018-12-18


path                  0
page_avg_scroll       0
page_avg_time         0
page_scroll_starts    0
page_total_time       0
page_uniques          0
page_views            0
page_views_loyal      0
page_views_quality    0
article_id            0
dia                   0
dtype: int64

['path', 'page_avg_scroll', 'page_avg_time', 'page_scroll_starts', 'page_total_time', 'page_uniques', 'page_views', 'page_views_loyal', 'page_views_quality', 'article_id', 'dia']
CONNECCION RAPIDA
2018-12-19
response:  {'ResponseMetadata': {'RequestId': '775A133563D54A45', 'HostId': 'nDk19rSWwWZYK4wmE3PnnFvoL1PGqhYsTdyBf04i2xESX8aSTFo8FkSLkYpQ6bi2YENoB3ThsAk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'nDk19rSWwWZYK4wmE3PnnFvoL1PGqhYsTdyBf04i2xESX8aSTFo8FkSLkYpQ6bi2YENoB3ThsAk=', 'x-amz-request-id': '775A133563D54A45', 'date': 'Fri, 08 Mar 2019 17:57:50 GMT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


,path,page_avg_scroll,page_avg_time,page_scroll_starts,page_total_time,page_uniques,page_views,page_views_loyal,page_views_quality,article_id,dia
0,archivo.gestion.pe/clinicas-lanzan-seguro-priv...,1562,19,1,19,1,1,0,1,44,2018-12-19
1,archivo.gestion.pe/designan-feliciano-amado-co...,1639,33,1,33,1,1,0,1,65,2018-12-19
2,archivo.gestion.pe/economia-peru-superara-chil...,1548,54,1,163,3,3,0,3,2050,2018-12-19
3,archivo.gestion.pe/economia/15-mujeres-mas-pod...,507,15,0,182,1,12,0,6,2167702,2018-12-19
4,archivo.gestion.pe/economia/alfredo-thorne-vue...,816,16,0,33,1,2,0,1,2187915,2018-12-19


...

,path,page_avg_scroll,page_avg_time,page_scroll_starts,page_total_time,page_uniques,page_views,page_views_loyal,page_views_quality,article_id,dia
0,archivo.gestion.pe/aje-preve-instalar-tres-pla...,949,25,1,25,1,1,0,1,2020,2019-01-17
1,archivo.gestion.pe/amp/panelg/retail-2019-como...,0,16,0,16,1,1,0,1,2209613,2019-01-17
2,archivo.gestion.pe/clinicas-lanzan-seguro-priv...,766,96,0,193,2,2,0,2,44,2019-01-17
3,archivo.gestion.pe/economia/25-anos-apuesta-pe...,300,0,0,1,2,2,0,0,2128305,2019-01-17
4,archivo.gestion.pe/economia/atencion-exportado...,0,10,0,10,1,1,0,0,2199148,2019-01-17


path                  0
page_avg_scroll       0
page_avg_time         0
page_scroll_starts    0
page_total_time       0
page_uniques          0
page_views            0
page_views_loyal      0
page_views_quality    0
article_id            0
dia                   0
dtype: int64

['path', 'page_avg_scroll', 'page_avg_time', 'page_scroll_starts', 'page_total_time', 'page_uniques', 'page_views', 'page_views_loyal', 'page_views_quality', 'article_id', 'dia']
CONNECCION RAPIDA
2019-01-18
response:  {'ResponseMetadata': {'RequestId': '9F35AADFF0A0BF4F', 'HostId': 'FMTipI3v7coxneUTSMB9LDYAvtxYFE1Qe4vio/SngEH3Vfbp4pdlkI+teZwvFcwFL39vrzgKM+4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'FMTipI3v7coxneUTSMB9LDYAvtxYFE1Qe4vio/SngEH3Vfbp4pdlkI+teZwvFcwFL39vrzgKM+4=', 'x-amz-request-id': '9F35AADFF0A0BF4F', 'date': 'Fri, 08 Mar 2019 18:05:17 GMT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


,path,page_avg_scroll,page_avg_time,page_scroll_starts,page_total_time,page_uniques,page_views,page_views_loyal,page_views_quality,article_id,dia
0,archivo.gestion.pe/ajegroup-venezuela-logro-is...,0,0,0,0,1,1,0,0,9001,2019-01-18
1,archivo.gestion.pe/economia/25-anos-inti-al-nu...,0,0,0,0,2,2,0,0,2129049,2019-01-18
2,archivo.gestion.pe/economia/53-cree-que-gobier...,0,1,0,1,1,1,0,0,2204067,2019-01-18
3,archivo.gestion.pe/economia/asi-funcionara-hid...,365,13,1,13,1,1,0,0,2199543,2019-01-18
4,archivo.gestion.pe/economia/cofopri-incrementa...,2,23,1,23,1,1,0,1,2169746,2019-01-18


path                  0
page_avg_scroll       0
page_avg_time         0
page_scroll_starts    0
page_total_time       0
page_uniques          0
page_views            0
page_views_loyal      0
page_views_quality    0
article_id            0
dia                   0
dtype: int64